In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import sys
import time
import h5py
sys.path.insert(1, '/home/nicolas/code/src')
sys.path.insert(1, '/home/nicolas/code/data')

from TMCRBM import TMCRBM
from scipy.integrate import simps
from scipy import stats


device = torch.device("cuda")
dtype = torch.float
torch.set_num_threads(4)
data = torch.tensor(np.genfromtxt('../dataset/data_2d.dat'), device = device, dtype = dtype)

data = (data+1)/2

In [2]:
f = h5py.File("../model/AllParametersTMC2DRBM_NGibbs_30_Nh100_Nv1000_Nmb200_Nepoch100_lr_0.1_N20000_Npoint2500_Nchain10.h5", 'r')
alltimes = np.array(f['alltime'])
alltimes = []
for t in f['alltime'][:]:
    if 'W'+str(t) in f:
        alltimes.append(t)

In [ ]:
S = torch.zeros(10, len(alltimes), device = device)
for i in range(len(alltimes)):
    t = alltimes[i]
    _, tmpS, tmpV = torch.svd(torch.tensor(f['W'+str(t)], device = device))
    if torch.mean(tmpV[:,0])<0:
        tmpV = -tmpV
    #plt.plot(tmpV[:,0].cpu(), zorder=0)
    S[:,i] = tmpS[:10]
#plt.hlines(1/np.sqrt(1000), 0, 1000, color = 'black', zorder=10)
plt.plot(alltimes,S.T.cpu())
plt.xlabel("nb_upd");

In [ ]:
print(alltimes)

In [ ]:
t = alltimes[-1]
lr = 0.1
NGibbs = 100
annSteps = 0
mb_s = 600
num_pcd = 600
Nh = torch.tensor(f['W0']).shape[0]
Nv = data.shape[1]
ep_max = 10
w_hat = torch.linspace(0,1,steps=100)
_, S_d, V = torch.svd(data)
V0 = V[:,0]
N = 20000
it_mean = 50
myRBM = TMCRBM(num_visible=Nv,
            num_hidden=Nh,
            device=device,
            lr=lr,
            gibbs_steps=NGibbs,
            UpdCentered=False,
            mb_s=mb_s,
            num_pcd=num_pcd,
            N = N,
            it_mean = it_mean,
           )

myRBM.W = torch.tensor(f['W'+str(t)], device = myRBM.device)
myRBM.hbias = torch.tensor(f['hbias'+str(t)], device = myRBM.device)
myRBM.vbias = torch.tensor(f['vbias'+str(t)], device = myRBM.device)
_, _, V = torch.svd(myRBM.W)
if torch.mean(V[:,0])<0:
    V = -V
f.close()

In [ ]:
# w_hat : nDim x nb_point
# V : matrice de projection
def TMCSampleND(v, w_hat, N, V, it_mcmc=100, it_mean=50, ß=1):
    #print("Initialisation")
    vtab = torch.zeros(v.shape, device=device)
    v_curr = v
    norm = 1/(v_curr.shape[0]**0.5)
    w_curr = (torch.mm(v_curr.T, V)*norm)[:,:w_hat.shape[0]]
    index = torch.randperm(v_curr.shape[0])
    for t in range(it_mcmc):
        #print('init it')
        print(t)
        h_curr, _ = myRBM.SampleHiddens01(v_curr)
        h_i = (torch.mm(myRBM.W.T, h_curr)+myRBM.vbias.reshape(v.shape[0],1)) # Nv x Ns
        w_next = w_curr.clone()
        
        v_next = torch.clone(v_curr)
        index = torch.randperm(v_curr.shape[0])
        for idx in range(v_curr.shape[0]):
            s = time.time()
            i = idx
            v_next[i,:] = 1-v_curr[i,:]
            for j in range(w_next.shape[1]):
                w_next[:,j] += ((2*v_next[i,:]-1)*V[i,j]*norm)
                
            # On calcul -DeltaE
            ΔE = ß*((2*v_next[i,:]-1)*h_i[i,:])-(N/2)*(torch.sum((w_hat.T-w_next)**2, dim=1)-torch.sum((w_hat.T-w_curr)**2, dim=1))

            tir = torch.rand(v_curr.shape[1],1, device = torch.device("cuda")).squeeze()
            prob = torch.exp(ΔE).squeeze()
            v_curr[i,:] = torch.where(tir<prob, v_next[i,:], v_curr[i,:])
            v_next[i,:] = torch.where(tir<prob, v_next[i,:], 1-v_next[i,:])
            neg_index = torch.ones(w_curr.shape[0], dtype = bool)
            index = torch.where(tir<prob)[0]
            neg_index[index] = False
            w_curr[index,:]=  w_next[index, :]
            w_next[neg_index,:] =  w_curr[neg_index,:]
            #print(time.time()-s)
        if (t>= (it_mcmc-it_mean)):
            vtab += v_curr
    vtab = vtab*(1/it_mean)
    vtab = vtab.reshape(Nv, nb_point, nb_chain)
    v_curr = v_curr.reshape(Nv, nb_point, nb_chain)
    h_curr = h_curr.reshape(Nh, nb_point, nb_chain)
    return v_curr, h_curr, vtab

In [ ]:
_, s, V_g = torch.svd(myRBM.W)
if torch.mean(V_g[:,0])<0:
    V_g = -V_g
proj_data =torch.mm(data, V_g).cpu()/Nv**.5


In [ ]:
print(s[:4])

In [ ]:
nb_chain = 10 # Nb de chaines pour chaque w_hat
it_mcmc = 50 # Nb it_mcmc pour chaque chaine
it_mean = 40 # Nb it considérée pour la moyenne temporelle de chaque chaine
nDim = 2
nb_point_dim = torch.tensor([50,50]) # -> doit avoir autant de terme que la valeur de nDim
N = 20000 # Contrainte
#nb_point_x = 125 # Nb de points de discrétisation pour w_hat
#nb_point_y = 125
width_plus = .4
limits = torch.zeros((2, nDim))
for i in range(nDim):
    limits[0, i] = proj_data[:,i].min()-width_plus
    limits[1, i] = proj_data[:,i].max()+width_plus

xmin = proj_data[:,0].min()-width_plus
xmax = proj_data[:,0].max()+width_plus
ymin = proj_data[:,1].min()-width_plus
ymax = proj_data[:,1].max()+width_plus
nb_point = nb_point_dim.prod()

In [ ]:
x_grid = np.linspace(limits[0,0], limits[1,0], nb_point_dim[0])
x_grid = np.array([x_grid for i in range(nb_point_dim[1])])
x_grid = x_grid.reshape(nb_point)
y_grid = []
y_d = np.linspace(limits[0,1], limits[1,1], nb_point_dim[1])
for i in range(nb_point_dim[1]):
    for j in range(nb_point_dim[0]):
        y_grid.append(y_d[i])
grid = torch.tensor([x_grid, y_grid], device = device, dtype = dtype)
grid.shape

In [ ]:
start = torch.bernoulli(torch.rand(myRBM.Nv, nb_chain*nb_point, device = device))
# w_hat = torch.dot(start.T, V)[0:,]
w_hat_b = grid
w_hat = torch.zeros((2, nb_chain*nb_point), device = device)
for i in range(nb_point):
    for j in range(nb_chain):
        w_hat[:,i*nb_chain+j] = w_hat_b[:,i]
tmpv, tmph, vtab = TMCSampleND(start, w_hat, N, V_g, it_mcmc = it_mcmc, it_mean=it_mean)



In [ ]:
vinit = torch.bernoulli(torch.rand(
        (myRBM.Nv, 10000), device=myRBM.device, dtype=myRBM.dtype))
#vinit = torch.where(V_g[:,0]>0, 1, 0).repeat(2000).reshape(myRBM.Nv, 2000).float()
#vinit = data.T
si, _, _, _ = myRBM.Sampling(vinit, it_mcmc=1000)
proj_gen = torch.mm(si.T, V_g).cpu().numpy()/Nv**.5
proj_data = torch.mm(data, V_g).cpu().numpy()/Nv**.5

In [ ]:
newy = torch.mm(torch.mean(vtab, dim = 2).T, V_g)[:,:nDim]/myRBM.Nv**0.5
#y = torch.mm(vtab.T, V_g)[:,:nDim]/myRBM.Nv**0.5
#newy = torch.tensor([torch.mean(y[i*nb_chain:i*nb_chain+nb_chain,:], dim = 0).cpu().numpy() for i in range(nb_point)])
#w_hat = w_hat.cpu().numpy()
#w_hat_b = w_hat_b.cpu().numpy()

In [ ]:
grad_pot = newy.T-w_hat_b
square = torch.zeros(2, nb_point_dim[0], nb_point_dim[1])
w_hat_tmp = np.zeros((2, nb_point_dim[0], nb_point_dim[1]))
for i in range(0,grad_pot.shape[1], nb_point_dim[0]):
        #print(\"I : \", i/nb_point_x)
        #print(\"s : \", (i*nb_point_x))
        #print(\"e : \", (i*nb_point_x+nb_point_x))
        w_hat_tmp[:,:,int(i/nb_point_dim[0])] = w_hat_b[:, i:(i+nb_point_dim[0])].cpu().numpy()
        #print(w_hat_b[:, i:(i+nb_point_dim[0])].cpu().numpy())
        square[:,:, int(i/nb_point_dim[0])] = grad_pot[:,i:(i+nb_point_dim[0])]
#square = grad_pot.reshape(2,nb_point_dim[0], nb_point_dim[1])
w_hat_dim = []
for i in range(nDim):
    w_hat_dim.append(np.linspace(limits[0,i], limits[1,i], nb_point_dim[i]))

res_x = np.zeros(nb_point_dim[0])
for i in range(nb_point_dim[0]):
    res_x[i] = simps(square[0][:(i+1),0].cpu().numpy(), w_hat_tmp[0,:(i+1),0])
res_y = np.zeros((nb_point_dim[0], nb_point_dim[1]))
for i in range(nb_point_dim[0]):
    for j in range(nb_point_dim[1]):
        res_y[i,j] = simps(square[1][i,:(j+1)].cpu().numpy(), w_hat_tmp[1,i,:(j+1)])

pot = np.expand_dims(res_x, 1).repeat(nb_point_dim[1],1) + res_y    
res = np.exp(N*(pot-np.max(pot)))

In [ ]:
#LE CALCUL DE LA CONSTANTE EST CORRECT
const = np.zeros(res.shape[0])
for i in range(res.shape[0]):
    const[i-1] = simps(res[:,i], w_hat_tmp[1, i, :])
const = simps(const, w_hat_tmp[0,:,0])
p_m = res/const

In [ ]:
tmp = np.zeros(p_m.shape[0])
for i in range( p_m.shape[0]):
    tmp[i] = simps(p_m[:,i], w_hat_dim[1])
tmp = simps(tmp, w_hat_dim[0])
print("Integrale de la proba sur l'espace : ", tmp)

In [ ]:
s_i = torch.mean(tmpv, dim = 2)
tau_a = torch.mean(tmph, dim = 2)
s_i_square = torch.zeros([s_i.shape[0], nb_point_dim[0], nb_point_dim[1]])
tau_a_square = torch.zeros([tau_a.shape[0], nb_point_dim[0], nb_point_dim[1]])

for i in range(0,grad_pot.shape[1], nb_point_dim[0]):
    s_i_square[:,:,int(i/nb_point_dim[0])] = s_i[:, i:(i+nb_point_dim[0])]
    tau_a_square[:,:,int(i/nb_point_dim[0])] = tau_a[:, i:(i+nb_point_dim[0])]
    

In [ ]:
prod = torch.zeros(
    (Nv, Nh, nb_point), device=device)
tmpcompute = torch.zeros(Nv, Nh, nb_chain)
s = time.time()
for i in range(nb_point):
    for k in range(nb_chain):
        tmpcompute[:,:,k] = torch.outer(tmpv[:, i, k], tmph[:, i, k])
    prod[:, :, i] = torch.mean(tmpcompute, dim = 2)
print("si tau_a prod : ", time.time()-s)
#prod = torch.stack([torch.mean(
#    prod[:, :, i*self.nb_chain:i*self.nb_chain+self.nb_chain], dim=2) for i in range(self.nb_point)], 2)

p_m = torch.tensor(p_m, device = device)
s_i_square = torch.zeros([s_i.shape[0], nb_point_dim[0], nb_point_dim[1]], device=device, dtype=dtype)
tau_a_square = torch.zeros([tau_a.shape[0], nb_point_dim[0], nb_point_dim[1]], device=device, dtype=dtype)
prod_square = torch.zeros((prod.shape[0], prod.shape[1], nb_point_dim[0], nb_point_dim[1]), device=device, dtype=dtype)
for i in range(0,grad_pot.shape[1], nb_point_dim[0]):
    s_i_square[:,:,int(i/nb_point_dim[0])] = s_i[:, i:(i+nb_point_dim[0])]
    tau_a_square[:,:,int(i/nb_point_dim[0])] = tau_a[:, i:(i+nb_point_dim[0])]
    prod_square[:,:,:,int(i/nb_point_dim[0])] = prod[:, :, i:(i+nb_point_dim[0])]

tmpres_s_i = torch.zeros(Nv, nb_point_dim[0], device=device, dtype=dtype)
tmpres_tau_a = torch.zeros(Nh, nb_point_dim[0], device=device, dtype=dtype)
tmpres_prod = torch.zeros((prod_square.shape[0], prod_square.shape[1], prod_square.shape[2]), device=device, dtype=dtype)
s_i_square = p_m*s_i_square #Ca fait bien ce qu'on veut
tau_a_square = p_m*tau_a_square
prod_square = p_m*prod_square
s_i_fin = torch.zeros(Nv, device=device, dtype=dtype)
tau_a_fin = torch.zeros(Nh, device=device, dtype=dtype)
prod_fin = torch.zeros(prod_square.shape[0], prod_square.shape[1], device=device, dtype=dtype)
for i in range(nb_point_dim[0]):
    tmpres_s_i[:,i] = torch.trapz(s_i_square[:,i,:], torch.tensor(w_hat_dim[1], device=device, dtype=dtype))
    tmpres_tau_a[:,i] = torch.trapz(tau_a_square[:,i,:], torch.tensor(w_hat_dim[1], device=device, dtype=dtype))
    tmpres_prod[:,:,i] = torch.trapz(prod_square[:,:,i,:], torch.tensor(w_hat_dim[1], device=device, dtype=dtype))
tau_a_fin = torch.trapz(tmpres_tau_a, torch.tensor(w_hat_dim[0], device=device, dtype=dtype))
s_i_fin = torch.trapz(tmpres_s_i, torch.tensor(w_hat_dim[0], device=device, dtype=dtype))
prod_fin = torch.trapz(tmpres_prod, torch.tensor(w_hat_dim[0], device=device, dtype=dtype))

In [ ]:
X, Y = np.meshgrid(w_hat_dim[0], w_hat_dim[1])

plt.figure(dpi=400)
plt.quiver(w_hat_tmp[0], w_hat_tmp[1], square[0].cpu(), square[1].cpu(), scale = 20)
plt.contour(w_hat_tmp[0],w_hat_tmp[1], torch.log(p_m.cpu()), 10)
plt.scatter(proj_data[:,0], proj_data[:,1])
plt.scatter(proj_gen[:,0], proj_gen[:,1])

In [ ]:
tmp.shape

In [ ]:
plt.imshow(p_m.cpu().T, extent = [xmin, xmax, ymin, ymax])
plt.colorbar()

In [ ]:
plt.hist(prod_fin.cpu().numpy().reshape(Nv*Nh), bins = 100);
plt.semilogy()

In [ ]:
X = myRBM.getMiniBatches(data.T, 15)
v_pos = X
h_pos_v, h_pos_m = myRBM.SampleHiddens01(X)

In [ ]:

X_pc, _, h_neg_v, h_neg_m = myRBM.GetAv()
_, _, h_neg_v_, h_neg_m_ = myRBM.GetAv()
v_neg = X_pc
v_neg_ = X_pc

h_neg_v_save = h_neg_v.clone()
NegTerm_ia = h_neg_v.mm(v_neg.t())
NegTerm_ia_bad = h_neg_v.mm(v_neg.t())
VisDataAv = torch.mean(v_pos, 1)
HidDataAv = torch.mean(h_pos_m, 1)
Xc_pos = (v_pos.t() - VisDataAv).t()
Hc_pos = (h_pos_m.t() - HidDataAv).t()

Xc_neg = (v_neg.t() - VisDataAv).t()
Hc_neg = (h_neg_m.t() - HidDataAv).t()

NormPos = 1.0/mb_s
NormNeg = 1.0/num_pcd
# NormL2 = self.regL2

siτa_neg = Hc_neg.mm(Xc_neg.t())*NormNeg
si_neg = torch.sum(v_neg, 1)*NormNeg
τa_neg = torch.sum(h_neg_m, 1)*NormNeg

ΔW = Hc_pos.mm(Xc_pos.t())*NormPos - siτa_neg

ΔW_neg_test = NegTerm_ia*NormNeg - torch.outer(torch.mean(h_neg_m,1).float(), VisDataAv) - \
            torch.outer(HidDataAv, torch.mean(v_neg,1)) + torch.outer(HidDataAv, VisDataAv)

ΔW_neg_test_bad = NegTerm_ia_bad*NormNeg - torch.outer(torch.mean(h_neg_v,1).float(), VisDataAv) - \
            torch.outer(HidDataAv, torch.mean(v_neg,1)) + torch.outer(HidDataAv, VisDataAv)

In [ ]:
plt.hist(siτa_neg.cpu().numpy().reshape(Nv*Nh), bins = 100);
# plt.hist(NegTerm_ia.cpu().numpy().reshape(Nv*Nh)*NormNeg, bins = 80,density=True);
plt.title("gradient normal");

In [ ]:
torch.mean(h_neg_v,1)

In [ ]:
plt.hist((ΔW_neg_test_bad).cpu().numpy().reshape(Nv*Nh), bins = 100);

In [ ]:
plt.plot(torch.abs(ΔW_neg_test_bad/ΔW_neg_test).cpu().numpy());
plt.semilogy()

In [ ]:
plt.hist(ΔW_neg_test.cpu().numpy().reshape(Nv*Nh), bins = 100);

In [ ]:
xxxx = (h_neg_v.mm(v_neg.t())/600 - h_neg_m.mm(v_neg.t())/600)

In [ ]:
y=torch.outer(torch.mean(h_neg_v,1).float(), VisDataAv) - torch.outer(torch.mean(h_neg_m,1).float(), VisDataAv)

In [ ]:
yy=torch.mean(h_neg_v,1).float() - torch.mean(h_neg_m,1).float()

In [ ]:
plt.hist(yy.reshape(100).cpu().numpy(),bins=50);

In [ ]:
plt.hist(y.reshape(100000).cpu().numpy(),bins=100);

In [ ]:
plt.hist(xxxx.reshape(100000).cpu().numpy(),bins=100);

In [ ]:
1/600**0.5

In [ ]:
print(v_neg.shape)
print(torch.mean(h_neg_v,1).shape)
VIS = VisDataAv.repeat(600,1).T

In [ ]:
(v_neg - VIS)

In [ ]:
Xc_neg

In [ ]:
_, _, V_d = torch.svd(data.T)

In [ ]:
w_hat_dim[0]

In [ ]:
plt.hist(ΔW.cpu().numpy().reshape(Nv*Nh), bins = 100);

In [ ]:
V_g.shape

In [ ]:
v_neg = s_i_fin
h_neg_v = tau_a_fin
h_neg_m = prod_fin.T
VisDataAv = torch.mean(v_pos, 1).float()
HidDataAv = torch.mean(h_pos_m, 1).float()    

NormPos = 1.0/mb_s
#NormNeg = 1.0/self.num_pcd


Xc_pos = (v_pos.t() - VisDataAv).t()
Hc_pos = (h_pos_m.t() - HidDataAv).t()

si_neg = v_neg
τa_neg = h_neg_v
print(h_neg_v.shape)
print(VisDataAv.shape)
print(v_neg.shape)
print(HidDataAv.shape)
print(torch.outer(HidDataAv, VisDataAv).shape)
# ΔW_neg = h_neg_m - torch.outer(h_neg_v.float(), VisDataAv) - torch.outer(HidDataAv, v_neg) + torch.outer(HidDataAv, VisDataAv)
ΔW_neg = h_neg_m - torch.outer(torch.mean(h_neg_v_,1).float(), VisDataAv) - torch.outer(HidDataAv, torch.mean(v_neg_,1)) + torch.outer(HidDataAv, VisDataAv)
ΔW = Hc_pos.mm(Xc_pos.t())*NormPos - ΔW_neg


In [ ]:
h_neg_v

In [ ]:
plt.hist(NegTerm_ia.cpu().numpy().reshape(Nv*Nh)*NormNeg, bins = 80,density=True);
plt.hist(prod_fin.cpu().numpy().reshape(Nv*Nh), bins =100,density=True);

In [ ]:
plt.hist(ΔW_neg.cpu().numpy().reshape(Nv*Nh), bins = 100);
# plt.hist(prod_fin.cpu().numpy().reshape(Nv*Nh), bins =100,density=True);
plt.title("gradient TMC 2D");

In [ ]:
plt.hist(siτa_neg.cpu().numpy().reshape(Nv*Nh), bins = 100);
# plt.hist(NegTerm_ia.cpu().numpy().reshape(Nv*Nh)*NormNeg, bins = 80,density=True);
plt.title("gradient normal");

In [ ]:
plt.hist(ΔW_neg_test.cpu().numpy().reshape(Nv*Nh), bins = 100);

In [ ]:
plt.hist(torch.mean(X_pc, 1).cpu().numpy(), density = True, bins = 100, label = "normal");
plt.hist(s_i_fin.cpu().numpy(), density = True, bins = 100, label = "TMC");
plt.legend()
plt.title("negTermV")

In [ ]:
plt.plot(s_i_fin.cpu(), label = "TMC")
plt.plot(torch.mean(X_pc, 1).cpu().numpy(), label = "normal")

plt.legend()
plt.title("negTermV")

In [ ]:
plt.hist(torch.mean(h_neg_v_save, 1).cpu().numpy(), label = "normal", bins =100)
plt.hist(tau_a_fin.cpu().numpy(), label = "TMC", bins =100)
plt.legend()
plt.title("negTermH")

In [ ]:
plt.plot(torch.mean(h_neg_v_save, 1).cpu().numpy(), label = "normal")
plt.plot(tau_a_fin.cpu().numpy(), label = "TMC")
plt.legend()
plt.title("negTermH")

In [ ]:
s_i.shape

In [ ]:
for k in range(100):
    print('-------------- : ', torch.mm(s_i[:,k].unsqueeze(1).T, V_g)[:,0])
    for i in range(10):
        proj = torch.mm(tmpv[:,k,i].unsqueeze(1).T, V_g)
        print(proj[:,0])
